In [23]:
import builtins
import contextlib
import io
import math
import os
from typing import Any
import requests
import json
from langchain_openai import ChatOpenAI
from langchain_openai import ChatOpenAI
from langgraph_codeact import create_codeact

import numpy as np
import pandas as pd
import scipy
import sklearn

from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENROUTER_API_KEY")

exec_globals = builtins.__dict__.copy()
exec_globals.update({
    "np": np,
    "pd": pd,
    "scipy": scipy,
    "sklearn": sklearn,
    "math": math,
})

def eval_code(code: str, context: dict[str, Any]) -> tuple[str, dict[str, Any]]:
    """
    A safe and enhanced code evaluation function to execute generated Python code.
    It uses traceback for detailed error reporting and handles variable scope correctly.
    """
    import traceback

    # The execution scope starts with the globally available libraries.
    exec_scope = exec_globals.copy()
    # It is then updated with the context from previous turns.
    exec_scope.update(context)

    stdout_io = io.StringIO()
    try:
        with contextlib.redirect_stdout(stdout_io):
            # Execute the code in the prepared scope.
            # New variables will be added to exec_scope.
            exec(code, exec_scope)
        
        output = stdout_io.getvalue()
        if not output:
            output = "<code ran, no output printed to stdout>"

    except Exception:
        # Capture the full traceback on error.
        output = f"Error during execution:\n{traceback.format_exc()}"

    # The exec_scope now contains the state after execution.
    # We filter out any non-serializable types (like class definitions)
    # before returning the context for the next turn.
    context_after_exec = {
        k: v for k, v in exec_scope.items() if not isinstance(v, type)
    }
    
    return output, context_after_exec

llm = ChatOpenAI(
        openai_api_base="https://openrouter.ai/api/v1",
        openai_api_key=os.getenv("OPENROUTER_API_KEY"),
        model="google/gemini-2.5-pro",
        streaming=False,
        max_completion_tokens=20000,
        request_timeout=120,
    )

In [38]:
from tools.tools import read_atoms_object, md_run_atoms

atoms = read_atoms_object('notebooks/test_slab.xyz')

md_run_atoms(atoms)

Using Materials Project MACE for MACECalculator with /home/vu/.cache/mace/20231203mace128L1_epoch199model
Using float32 for MACECalculator, which is faster but less accurate. Recommended for MD. Use float64 for geometry optimization.


/home/vu/miniconda3/envs/llm_adsorbate/lib/python3.10/site-packages/mace/calculators/mace.py:197: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)


Using head Default out of ['Default']
Default dtype float32 does not match model dtype float64, converting models to float32.


RuntimeError: Atoms object has no calculator.